In [3]:
import time
from pymoo.algorithms.moo.nsga2 import NSGA2
from genetic_algorithm import main_algorithm

import orders
from genetic_helperfunctions import makespan


### nochmal schauen ob das bei init_df die auswahl des tails sinn ergibt

In [ ]:
id_length = len(orders.get_westaflex_orders())
ids=[]
for i in range (id_length):
    x = id_length-(i+1)
    ids.append(x)



In [ ]:
starttime = time.time()
print("Start")
x = makespan(ids)
endtime = time.time()
print(endtime - starttime)
print(x)

 ## Genetischer Algorithmus

In [ ]:
starttime = time.time()

output = main_algorithm()

endtime = time.time()

print(endtime - starttime)

In [ ]:
print(output[1])

## Vergleich der Algorithmen

In [1]:
from comparison_functions import makespan_short, avg_lateness_short
from genetic_helperfunctions import initializing_new_df
from termination import naive_termination
from pyomomachsched import opt_schedule
import datetime

In [4]:
id_length = len(orders.get_westaflex_orders())
ids=[]
for i in range (id_length):
    x = i
    ids.append(x)

In [ ]:
print(ids)

In [5]:
df = initializing_new_df(ids)
df_short = df.tail(n=35)
df_short = df.reset_index(drop=True)


In [ ]:
starttime = time.time()
print("Start")
#df_naive = naive_termination(df_short, datetime.datetime(2022, 2, 27, 6, 0, 0), "A0")
df_pyomo = opt_schedule(df_short, datetime.datetime(2022, 1, 2, 0, 0, 0))
endtime = time.time()
print(endtime - starttime)

Start
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
11 Set Declarations
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  120 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119}
    JM : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain    : Size : Members
        None :     2 : JM_domain :  320 : {(75, '1533'), (84, '1531'), (95, '1531'), (84, '1536'), (11, '1537'), (60, '1535'), (57, '1536'), (8, '1531'), (19, '1531'),

        Key                : Lower : Value : Upper : Fixed : Stale : Domain
            (0, 1, '1531') :     0 :  None :     1 : False :  True : Binary
            (0, 1, '1532') :     0 :  None :     1 : False :  True : Binary
            (0, 1, '1533') :     0 :  None :     1 : False :  True : Binary
            (0, 2, '1531') :     0 :  None :     1 : False :  True : Binary
            (0, 2, '1532') :     0 :  None :     1 : False :  True : Binary
            (0, 2, '1533') :     0 :  None :     1 : False :  True : Binary
            (0, 4, '1531') :     0 :  None :     1 : False :  True : Binary
            (0, 4, '1532') :     0 :  None :     1 : False :  True : Binary
            (0, 4, '1533') :     0 :  None :     1 : False :  True : Binary
            (0, 5, '1531') :     0 :  None :     1 : False :  True : Binary
            (0, 5, '1532') :     0 :  None :     1 : False :  True : Binary
            (0, 5, '1533') :     0 :  None :     1 : False :  True : Binary
            

In [ ]:
#nur einmal nutzen, da sonst 
makespan_naive = makespan_short(df_naive)
lateness_naive = avg_lateness_short(df_naive)

print("naive")
print(makespan_naive)
print(lateness_naive)


In [ ]:
makespan_pyomo = makespan_short(df_pyomo)
lateness_pyomo = avg_lateness_short(df_pyomo)


print("pyomo")
print(makespan_pyomo)
print(lateness_pyomo)